In [ ]:
from CNN.cifar_model import MobileNet
import torch
import torch.nn as nn
import time
from torchvision import datasets, transforms
import quartz
from quartz import layers
import copy
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
first_layer = 0
last_layer = 7

In [ ]:
model = MobileNet(10)
model.load_state_dict(torch.load("CNN/models/cifar-convnet.pth", map_location=torch.device('cpu')))
capture = model.eval()

In [ ]:
previous_module = None
new_layers = []
for module in model.modules():
    if isinstance(module, (nn.Conv2d, nn.MaxPool2d, nn.BatchNorm2d, nn.Linear, nn.ReLU6, nn.ReLU, nn.Flatten)):
        if isinstance(module, nn.BatchNorm2d) and isinstance(previous_module, nn.Conv2d):
            new_layers[-1] = torch.nn.utils.fuse_conv_bn_eval(previous_module, module)
        else:
            new_layers.append(module)
        previous_module = module

folded_model = nn.Sequential(*new_layers)
layer_list = list(folded_model.modules())[1:]

In [ ]:
t_max = 2**6
input_dims = (3,32,32)
pool_kernel_size = [2,2]

loihi_layers = []
for l, layer in enumerate(layer_list):
    rectification = l < len(layer_list)-1 and isinstance(layer_list[l+1], (nn.ReLU6, nn.ReLU))
    if isinstance(layer, nn.Conv2d):
        loihi_layers.append(layers.Conv2D(weights=layer.weight.detach().numpy(), biases=layer.bias.detach().numpy(), stride=layer.stride, padding=layer.padding, groups=layer.groups, rectifying=rectification))
    elif isinstance(layer, nn.Linear):
        loihi_layers.append(layers.Dense(weights=layer.weight.detach().numpy(), biases=layer.bias.detach().numpy(), rectifying=rectification))
    elif isinstance(layer, nn.MaxPool2d):
        loihi_layers.append(layers.MaxPool2D(kernel_size=layer.kernel_size, stride=layer.stride))

all_layers = [layers.InputLayer(dims=input_dims)] + copy.deepcopy(loihi_layers)

In [ ]:
loihi_model = quartz.Network(t_max, all_layers[:first_layer+1], verbose=True)

In [ ]:
loihi_layers

In [ ]:
len(loihi_layers)

In [ ]:
loihi_layers[first_layer:last_layer]

## see if the thing compiles without errors, otherwise reduce #neurons on that layer

In [ ]:
n_cores_per_layer = [350]*(last_layer - first_layer + 1)
n_cores_per_layer = [0, 300, 150, 175]

In [ ]:
input_dims = loihi_model.layers[first_layer].output_dims

selected_layers = [layers.InputLayer(dims=input_dims)] + copy.deepcopy(loihi_layers[first_layer:last_layer])
layer_model = quartz.Network(t_max, selected_layers, verbose=False)
try:
    layer_model(np.random.rand(*input_dims), n_cores_per_layer=n_cores_per_layer, partition='nahuku32', profiling=True, logging=True)
except Exception as e: print(e)